In [1]:
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import luno_python
import krakenex
from pykrakenapi import KrakenAPI
from forex_python.converter import CurrencyRates
from datetime import datetime
import time
# import pyokx
from binance.client import Client

import sys
sys.path.append("/Users/chunkiat/Documents/Python/Crypto_trading/crypto_trading_system/")
from src.brokerage.luno.TradeClient import TradeClient as tcl
from src.brokerage.kraken.TradeClient import TradeClient as tck
from src.brokerage.binance.TradeClient import TradeClient as tcb
from src.events_engine import EventEngine
from src.events import (
    EventType,
    TickEvent,
    OrderEvent,
    CheckOrderStatusEvent,
    FillEvent,
)

In [9]:
my_config = {
    "luno_key_id": "jyfg7e2nfy9y8",
    "luno_key_secret": "hYtkGho0RgerI7SLKSjvFJnJaXoL22dj7078UwiHrBA"
}

my_config = {
    "luno_key_id": "e8eszm4zn6ruu",
    "luno_key_secret": "HvYuGeYG8_lOl1-M1a6hZ07ZaB9oAE2lGNFuC2S6-mQ"
}

kraken_config = {
    "kraken_key_id": "k1Vffnd5kXKlN9BT7dfq2AkSWjJioczk2LVH6bAN0srIJ9qW7tIk/+RE",
    "kraken_private_key": "1MZ9mgBnG0TsDdDf+HJJePCqI7cjRqOJHdNosLlX2YETQYTdNyLXIBme+PdMmK45VQyAUslp63dZaoJFSFBzbA==",
}

tcl1 = tcl(auth_config=my_config, mkt_event_engine= EventEngine(), action_event_engine = EventEngine())

In [16]:
coin = "ETH"
df_luno = tcl1.get_historical_ohlcv_data(instrument =  f"{coin}MYR", 
                                        start = "2022-07-01", 
                                        end = "2023-06-30", 
                                        granularity=60)
df_luno

,date,open,close,high,low,volume
0,2022-07-01 00:00:00,4721.0,4711.0,4721.0,4711.0,0.6856
1,2022-07-01 00:01:00,4711.0,4705.0,4713.0,4697.0,8.5358
2,2022-07-01 00:02:00,4696.0,4662.0,4696.0,4662.0,14.0817
3,2022-07-01 00:03:00,4663.0,4660.0,4663.0,4660.0,2.4580
4,2022-07-01 00:04:00,4662.0,4589.0,4662.0,4589.0,10.0143
...,...,...,...,...,...,...
524995,2023-06-30 13:55:00,8596.0,8600.0,8600.0,8596.0,1.0058
524996,2023-06-30 13:56:00,8612.0,8610.0,8612.0,8600.0,0.4475
524997,2023-06-30 13:57:00,8610.0,8610.0,8610.0,8610.0,0.0000
524998,2023-06-30 13:58:00,8612.0,8603.0,8612.0,8602.0,0.1539


In [4]:
df_luno.to_csv(f"./luno_{coin}MYR_one_year.csv", index = False)

In [3]:
from luno_python.client import Client
id =my_config["luno_key_id"]
secret =my_config["luno_key_secret"]
client = Client(api_key_id=id, api_key_secret=secret)
df= pd.DataFrame(client.get_candles(3600, "UNIMYR",tcl1.format_date("2023-05-08"))["candles"])
df.columns = ["date", "open", "close", "high", "low", "volume"]
# df["date"] = pd.to_datetime(df["date"], unit="ms")
# df["date"] = pd.Timestamp(df["date"], unit="ms")
df["date"] = (df["date"]//1000).apply(datetime.fromtimestamp)
df["date"]

0     2023-05-08 00:00:00
1     2023-05-08 01:00:00
2     2023-05-08 02:00:00
3     2023-05-08 03:00:00
4     2023-05-08 04:00:00
              ...        
204   2023-05-16 12:00:00
205   2023-05-16 13:00:00
206   2023-05-16 14:00:00
207   2023-05-16 15:00:00
208   2023-05-16 16:00:00
Name: date, Length: 209, dtype: datetime64[ns]

In [5]:
tcb1 = tcb(mkt_event_engine= EventEngine(), action_event_engine = EventEngine())

In [6]:
df_binance = tcb1.get_historical_ohlcv_data(instrument =  f"{coin}USDT", 
                                            start = "2022-07-01", 
                                            end = "2023-06-30", 
                                            granularity=60)

In [7]:
df_binance.to_csv(f"./binance_{coin}USD_one_year.csv", index = False)


In [ ]:
INTERVAL = {
    60 : "1m",
    180 : "3m",
    300 : "5m",
    900 : "15m",
    1800 : "30m",
    3600 : "1h",
    7200 : "2h",
    14400 : "4h",
    21600 : "6h",
    28800 : "8h",
    43200 : "12h",
    86400 : "1d",
    259200 : "3d",
    604800: "1w"
}


In [9]:
pd.to_datetime("2023-04-30")

Timestamp('2023-04-30 00:00:00')

In [ ]:
c = CurrencyRates()
date_obj = datetime(2023,3,30)
c.get_rate("USD", "MYR", date_obj=date_obj)

In [ ]:
df = pd.DataFrame(client.get_klines(symbol='UNIUSDT', interval="1m"))
df = df.iloc[:,:5]
numeric_columns = ["open_bin", "high_bin", "low_bin", "close_bin"]
df.columns = ["date"] + numeric_columns
df["date"] = pd.to_datetime(df["date"], unit = "ms")
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors = "coerce")
df

## Kraken API

In [ ]:
api = krakenex.API()
k = KrakenAPI(api)
ohlc, last = k.get_ohlc_data("UNIUSD", since=)
ohlc = pd.DataFrame(ohlc).reset_index()
ohlc = ohlc[["dtime", "open", "high", "low", "close"]]
ohlc.columns = ["date","open_krk", "high_krk", "low_krk", "close_krk"]
ohlc


In [ ]:
df1 = df.merge(ohlc, on="date", how = "inner")
df1["open_diff"] = (df1["open_bin"] - df1["open_krk"])/df1["open_bin"]*100
df1["high_diff"] = (df1["high_bin"] - df1["high_krk"])/df1["high_bin"]*100
df1["close_diff"] = (df1["close_bin"] - df1["close_krk"])/df1["close_bin"]*100
df1["low_diff"] = (df1["low_bin"] - df1["low_krk"])/df1["low_bin"]*100
df1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df1.date, 
                         y = df1.open_diff, 
                         mode = "lines", 
                         name = "Open"))
fig.add_trace(go.Scatter(x = df1.date, 
                         y = df1.close_diff, 
                         mode = "lines", 
                         name = "Close"))
fig.add_trace(go.Scatter(x = df1.date, 
                         y = df1.high_diff, 
                         mode = "lines", 
                         name = "High"))
fig.add_trace(go.Scatter(x = df1.date, 
                         y = df1.low_diff, 
                         mode = "lines", 
                         name = "Low"))
fig.update_layout(title = "Pct difference b/w Binance & Kraken (UNIUSD)",
                  xaxis_title = "GMT",
                  yaxis_title = "%")
fig.show()

## Kraken Order Book



In [ ]:
import requests

resp = requests.get('https://api.kraken.com/0/public/Depth?pair=BTCUSD&count=2')

print(resp.json())


In [ ]:
kraken = krakenex.API()
kraken.query_public('Depth', {'pair': 'UNIUSD', 'count': '2', 'since':start_epoch})


In [ ]:
start_epoch

## Kraken Recent Trades

In [ ]:
symbol = "UNIUSD"
start = "2023-03-31"
end = "2023-04-01"
start_epoch = int(tck.format_date("2023-03-31")/1000) # in seconds
end_epoch = int(tck.format_date("2023-04-02")/1000) # in seconds
resp = requests.get(f'https://api.kraken.com/0/public/Trades?pair={symbol}&since={start_epoch}')
data = resp.json()["result"][symbol]

In [ ]:
df_json = pd.DataFrame(data)
df_json.columns = ["price", "volume", "date", "action", "order_type","miscellaneous", "trade_id"]
df_json["date"] = pd.to_datetime(df_json["date"], unit = "s")
df_json

## Bitfinex

In [ ]:
from  src.brokerage.bitfitnex.TradeClient import TradeClient as tcb # pip install bitfinex-api-py
from bfxapi import Client
from dateutil.relativedelta import relativedelta
import datetime
import pandas as pd
import time
from src.events import EventType, Exchange, TickEvent, OrderEvent
from src.events_engine import EventEngine

In [ ]:
# Needs to implement 
# 1. get ohlc (pair: str, count: int, granularity: int)
# 2. get ohlc history (instrument, start: str, end: str, granularity :int = 3600 )




In [ ]:
tcb(mkt_event_engine= EventEngine(), action_event_engine = EventEngine())

In [ ]:
numeric_columns = ["open", "close", "high", "low", "volume"]

symbol = "UNIUSD"

granularity = ["1m", "5m", "15m", "30m", "1h", "3h", "6h", "12h", "1D", "1W", "14D", "1M"]

start = "2023-03-31"
end = "2023-04-02"
start_epoch = int(tcb.format_date(start)) # in ms
end_epoch = int(tcb.format_date(end)) # in ms
df_bitfinex = pd.DataFrame()
while start_epoch < end_epoch:
    url = "https://api-pub.bitfinex.com/v2/candles/trade%3A5m%3AtBTCUSD/hist?start={start_epoch}&sort=+1" # sort = +1 for ascending order in ts
    resp = requests.get(url)
    data = resp.json()
    df_json = pd.DataFrame(data)
    df_json.columns = ["date"] + numeric_columns
    df_json["date"] = pd.to_datetime(df_json["date"], unit = "ms")
    df_json[numeric_columns] = df_json[numeric_columns].apply(pd.to_numeric, errors = "coerce")
    
    df_bitfinex = pd.concat([df_bitfinex , df_json], axis = 0)
    start_epoch = int(data[-1][0])
    time.sleep(0.1)

    
df_bitfinex = df_bitfinex.drop_duplicates(subset = ["date"])
# df_kraken = df_kraken[(df_kraken["date"] >= datetime.strptime(start, '%Y-%m-%d')) & 
#                       (df_kraken["date"] < datetime.strptime(end, '%Y-%m-%d'))]
df_bitfinex = df_bitfinex.reset_index(drop = True)
df_bitfinex



In [ ]:
len(df_bitfinex["date"].unique())

In [ ]:

start = "2023-05-03"
end = "2023-05-05"
start_epoch = int(tcb.format_date(start)) # in ms
end_epoch = int(tcb.format_date(end)) # in ms

symbol = "BTCUSD"
# url = "https://api-pub.bitfinex.com/v2/candles/trade%3A5m%3At{symbol}/hist?start={1683961680000}&sort=+1" # sort = +1 for ascending order in ts
url = "https://api-pub.bitfinex.com/v2/candles/trade%3A1m%3AtBTCUSD/hist?end={end_epoch}&sort=+1"
resp = requests.get(url)
data = resp.json()
data

In [ ]:
pd.DataFrame(resp.json())

In [ ]:
from pykrakenapi import KrakenAPI
import krakenex

In [ ]:
k= krakenex.API()
# k = KrakenAPI(api)

# Initial OHLC dataframe

k.query_public('OHLC', req = {'pair': "BTCUSD", 'since': str(start_epoch)})

In [ ]:
df

In [ ]:
start_epoch

In [ ]:
import ccxt

hitbtc   = ccxt.hitbtc({'verbose': True})
bitmex   = ccxt.bitmex()
huobipro = ccxt.huobipro()



In [ ]:
kraken = ccxt.kraken()

In [ ]:
data = kraken.fetch_ohlcv(symbol = "BTC/USD", since = start_epoch)
df_json = pd.DataFrame(data)
df_json.columns = ["date"] + numeric_columns
df_json["date"] = pd.to_datetime(df_json["date"], unit = "ms")
df_json[numeric_columns] = df_json[numeric_columns].apply(pd.to_numeric, errors = "coerce")


In [2]:
import pandas as pd
import datetime as dt
from binance.client import Client


INTERVAL = {
    60 : "1m",
    180 : "3m",
    300 : "5m",
    900 : "15m",
    1800 : "30m",
    3600 : "1h",
    7200 : "2h",
    14400 : "4h",
    21600 : "6h",
    28800 : "8h",
    43200 : "12h",
    86400 : "1d",
    259200 : "3d",
    604800: "1w"
}

client=Client()

def get_historical_ohlc_data(instrument: str ,start: str, end: str, granularity: int = 3600
) -> pd.DataFrame:
    # start_ts = tcb.format_date(start)
    # end_ts = tcb.format_date(end)

    """Returns historcal klines from past for given symbol and interval
    past_days: how many days back one wants to download the data"""

    if granularity not in INTERVAL.keys():
        raise NotImplementedError(
            "Selected granularity not supported in python-binance API"
        )
    # start_str=str((pd.to_datetime('today')-pd.Timedelta(str(past_days)+' days')).date())

    numeric_columns = ["open", "close", "high", "low", "volume"]
    res=pd.DataFrame(client.get_historical_klines(symbol=instrument, start_str = start, interval=INTERVAL[granularity]))
    res.columns=['date','open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol','is_best_match']
    res['date']=[dt.datetime.fromtimestamp(x/1000) for x in res["date"]]
    res = res[res["date"] < pd.to_datetime("2023-05-05")]
    # print(pd.to_datetime(end, unit="ms"))
    res = res.drop_duplicates(subset = ["date"])
    res['symbol']=instrument
    res[numeric_columns] = res[numeric_columns].apply(
        pd.to_numeric, errors="coerce")
    res = res.reset_index(drop=True)
    return res[["date"] + numeric_columns]


In [4]:
df = get_historical_ohlc_data(instrument= "BTCUSDT" ,start = "2023-05-02", end = "2023-05-05", granularity =  1800)
df

,date,open,close,high,low,volume
0,2023-05-02 08:00:00,28068.26,28075.01,28080.30,27978.11,1067.38222
1,2023-05-02 08:30:00,28075.02,28050.42,28145.60,28020.00,762.91203
2,2023-05-02 09:00:00,28050.42,28044.02,28080.07,28009.00,440.35766
3,2023-05-02 09:30:00,28044.02,27942.87,28092.61,27918.49,846.80587
4,2023-05-02 10:00:00,27942.86,27956.85,28035.98,27925.74,589.63127
...,...,...,...,...,...,...
123,2023-05-04 21:30:00,28881.09,28761.51,28914.16,28663.64,3428.16069
124,2023-05-04 22:00:00,28761.52,28860.01,28891.19,28736.33,1609.59396
125,2023-05-04 22:30:00,28860.00,28783.65,28938.54,28761.73,1451.98711
126,2023-05-04 23:00:00,28783.66,28894.53,28949.66,28774.58,1398.90846


In [ ]:
max(df["date"])

In [5]:
client.get_exchange_info()

{'timezone': 'UTC',
 'serverTime': 1684678693465,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelReplaceAllowed': True,
   'i

In [7]:
tcl1.get_ohlcv("UNIMYR", count = 120, granularity = 60)

1684678720911 1684671520911


,date,open,close,high,low,volume
0,2023-05-21 12:19:00,23.63,23.63,23.63,23.63,0.00
1,2023-05-21 12:20:00,23.63,23.63,23.63,23.63,0.00
2,2023-05-21 12:21:00,23.63,23.63,23.63,23.63,0.00
3,2023-05-21 12:22:00,23.63,23.63,23.63,23.63,0.00
4,2023-05-21 12:23:00,23.63,23.63,23.63,23.63,0.00
...,...,...,...,...,...,...
113,2023-05-21 14:12:00,23.66,23.66,23.66,23.66,0.00
114,2023-05-21 14:13:00,23.66,23.66,23.66,23.66,0.00
115,2023-05-21 14:14:00,23.66,23.66,23.66,23.66,0.00
116,2023-05-21 14:15:00,23.66,23.66,23.66,23.66,0.00


In [27]:
datetime.now()

datetime.datetime(2023, 5, 21, 22, 9, 4, 91430)

In [7]:
tcb1.get_ohlcv(pair = "UNIUSDT", granularity = 60)

,date,open,close,high,low,volume
0,2023-05-21 06:33:00,5.247,5.247,5.247,5.247,104.36
1,2023-05-21 06:34:00,5.246,5.243,5.246,5.243,445.59
2,2023-05-21 06:35:00,5.244,5.243,5.244,5.243,72.59
3,2023-05-21 06:36:00,5.243,5.244,5.244,5.243,651.47
4,2023-05-21 06:37:00,5.244,5.244,5.244,5.244,6.81
...,...,...,...,...,...,...
495,2023-05-21 14:48:00,5.204,5.204,5.204,5.204,0.00
496,2023-05-21 14:49:00,5.204,5.206,5.206,5.204,229.92
497,2023-05-21 14:50:00,5.206,5.208,5.210,5.206,724.64
498,2023-05-21 14:51:00,5.209,5.209,5.209,5.209,0.97


,date,open,close,high,low,volume
0,2023-05-21 06:01:00,5.258,5.258,5.258,5.258,2.99
1,2023-05-21 06:02:00,5.257,5.251,5.257,5.251,916.13
2,2023-05-21 06:03:00,5.251,5.252,5.252,5.251,218.46
3,2023-05-21 06:04:00,5.251,5.250,5.251,5.250,353.22
4,2023-05-21 06:05:00,5.251,5.252,5.252,5.251,57.12
...,...,...,...,...,...,...
495,2023-05-21 14:16:00,5.203,5.203,5.204,5.203,107.78
496,2023-05-21 14:17:00,5.202,5.202,5.202,5.202,10.00
497,2023-05-21 14:18:00,5.202,5.202,5.202,5.202,258.07
498,2023-05-21 14:19:00,5.202,5.204,5.204,5.202,319.61


In [18]:
time.time()

1685349326.789439

In [19]:
time.time_ns()

1685349331600595000

In [21]:
print(datetime(2023,10,9,18,0,0))

2023-10-09 18:00:00


In [22]:
print(datetime.fromisoformat("2023-09-16T14:05:13"))

2023-09-16 14:05:13


In [25]:
print(datetime.now())

2023-05-29 16:51:47.592960


In [27]:
from datetime import datetime
from pytz import timezone

some_date = datetime.fromisoformat("2023-09-16T14:05:13")

utc = timezone("UTC")
loc = utc.localize(some_date)


print(some_date)
print(loc)

sydney = timezone("Australia/Sydney")
print(loc.astimezone(sydney))

amsterdam = timezone("Europe/Amsterdam")
print(loc.astimezone(amsterdam))

2023-09-16 14:05:13
2023-09-16 14:05:13+00:00
2023-09-17 00:05:13+10:00
2023-09-16 16:05:13+02:00
